<a href="https://colab.research.google.com/github/uucad/aigc_LLM_engineering/blob/main/chatpdf_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 加载Google云硬盘
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install langchain
!pip install pypdf
!pip install InstructorEmbedding
!pip install sentence_transformers
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 18.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-pyth

In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

root_dir = '/content/drive/MyDrive'
# 读取所有的pdf文件
loader = DirectoryLoader(f'{root_dir}/PDFs/', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
print(documents)

[Document(page_content='  \n  \nDPDK Cookbook  \n \n  \n \n \n  \nFeaturing:  \nSolution -Oriented Mini Sections  \nUser -Friendly Screen shots  \nLinks to Videos and Online Contents  \n \nOverview  \n \nThe DPDK Cookbook modules teach  you everything you need to know to be productive with the Data Plane Develop ment  \nKit (DPDK) . Here’s an overview  of the topics covered : \n\uf0b7 Build Your Own DPDK Traffic Generator —DPDK -In-A-Box \n\uf0b7 DPDK Transmit and Receive —DPDK -in-a-Box \n\uf0b7 Build Your Own DPDK Packet Framework with DPDK -In-A-Box \n\uf0b7 DPDK Data Plane —Multicores an d Control Plane Synchronization  \n\uf0b7 DPDK Performance Optimization Guidelines White Paper  \n\uf0b7 Profiling DPDK Code with Intel® VTune™ Amplifier  \n\uf0b7 References  \nI highly recommend that you devour the Architecture Overview  section of the Programm er’s G uide at dpdk.org. This \nexcellent document, authored by architects and designers, goes into both the how and the why of DPDK desi

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 拆分长文本
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(texts[0])

page_content='DPDK Cookbook  \n \n  \n \n \n  \nFeaturing:  \nSolution -Oriented Mini Sections  \nUser -Friendly Screen shots  \nLinks to Videos and Online Contents  \n \nOverview  \n \nThe DPDK Cookbook modules teach  you everything you need to know to be productive with the Data Plane Develop ment  \nKit (DPDK) . Here’s an overview  of the topics covered : \n\uf0b7 Build Your Own DPDK Traffic Generator —DPDK -In-A-Box \n\uf0b7 DPDK Transmit and Receive —DPDK -in-a-Box \n\uf0b7 Build Your Own DPDK Packet Framework with DPDK -In-A-Box \n\uf0b7 DPDK Data Plane —Multicores an d Control Plane Synchronization  \n\uf0b7 DPDK Performance Optimization Guidelines White Paper  \n\uf0b7 Profiling DPDK Code with Intel® VTune™ Amplifier  \n\uf0b7 References  \nI highly recommend that you devour the Architecture Overview  section of the Programm er’s G uide at dpdk.org. This \nexcellent document, authored by architects and designers, goes into both the how and the why of DPDK design.' metadata={'so

In [7]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
# 设置embedding model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large",model_kwargs={"device": "cuda"})
# 将texts转化为embeddings，存放到Faiss中
db_instructEmbedd = FAISS.from_documents(texts, instructor_embeddings)
# 设置相似度查找方式，Topk=3
retriever = db_instructEmbedd.as_retriever(search_kwargs={"k": 3})
retriever.search_type

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


'similarity'

In [8]:
# 相似文档检索
docs = retriever.get_relevant_documents("How to use Trex?")
print(len(docs))
docs[0]

3


Document(page_content='You will see the file t-rex-64, which is the traffic generator executable:  \n \nConfigure the Traffic Generator  \nThe good news is that the TRex package comes with a sample config file cfg/simple_cfg.yaml . Copy that to \n/etc/trex_cfg.yaml  and edit the file by issuing the following commands, making sure that you’re in your \n/home/test/trex/<your version>  direct ory: \n# pwd \n# cp cfg/simple_cfg.yaml /etc/trex_cfg.yaml  \n# gedit /etc/trex_cfg.yaml', metadata={'source': '/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf', 'page': 16})

In [9]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.7 MB/s eta 0:00:00


In [10]:
import os
# os.environ["OPENAI_API_KEY"] = 'sk-'

from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

## LLM回答
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

# 使用Chain，回答问题
qa_chain_instrucEmbed = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0.2, ),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)


query = 'How to use Trex?'
print('-------------------Instructor Embeddings------------------\n')
llm_response = qa_chain_instrucEmbed(query)
process_llm_response(llm_response)

-------------------Instructor Embeddings------------------

 To use Trex, you need to configure the Traffic Generator with the command 'cp cfg/simple_cfg.yaml /etc/trex_cfg.yaml' and then install the Traffic Generator with the command 'wget –no-cache http://trex -tgn.cisco.com/trex/release/latest'. You can then run Trex with the parameters -f, -c, and -d.


Sources:
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf


In [11]:
query = "How to use pktgen"
print('-------------------Instructor Embeddings------------------\n')
llm_response = qa_chain_instrucEmbed(query)
process_llm_response(llm_response)

-------------------Instructor Embeddings------------------

 Pktgen is a traffic generator tool that is part of the DPDK suite of tools. It can be used to generate traffic for testing and benchmarking purposes. To use pktgen, you need to install the DPDK and then configure the pktgen application with the desired packet length, number of packets to send, and other parameters. You can then run the pktgen application to generate traffic. To measure latency, you can use the Intel VTune Amplifier tool.


Sources:
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf


In [12]:
query = "How to use dpdk-testpmd"
print('-------------------Instructor Embeddings------------------\n')
llm_response = qa_chain_instrucEmbed(query)
process_llm_response(llm_response)

-------------------Instructor Embeddings------------------

 To use dpdk-testpmd, first run the command ./x86_64-native-linuxapp-gcc/app/testpmd -- -i to start testpmd. This command is also available as a script named run.sh in DPDK-in-a-Box. Initialization consists of three steps: 1. EAL (Environment Abstraction Layer) - Find the number of cores and probe PCI devices. Once initialized, testpmd can be used to evaluate a DPDK application using PCM tools such as PCM-x, PCM-memory, and PCM-iio.


Sources:
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf


In [13]:
query = "How to install and configure dpdk-testpmd"
print('-------------------Instructor Embeddings------------------\n')
llm_response = qa_chain_instrucEmbed(query)
process_llm_response(llm_response)

-------------------Instructor Embeddings------------------

 The initial portion of testpmd initializes the Environment Abstraction Layer (EAL) to find the number of cores and probe PCI devices. The runtime portion of testpmd is used to run the testpmd application. To install and configure dpdk-testpmd, you can use the setup.sh script in the tools subdirectory of the DPDK install, the dpdk_nic_bind.py script in the tools subdirectory, the cpu_layout.py script in the tools subdirectory, the pmdinfo.py script in the tools subdirectory, and the DPDK Test Suite (DTS) Getting Started Guide.


Sources:
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf


In [14]:
query = "How to configure VLAN strip in DPDK?"
print('-------------------Instructor Embeddings------------------\n')
llm_response = qa_chain_instrucEmbed(query)
process_llm_response(llm_response)

-------------------Instructor Embeddings------------------

 VLAN strip is not configured in DPDK. It is configured in the network interface controller driver. Please refer to the driver documentation for more information.


Sources:
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf


In [15]:
query = "How to check dpdk-testpmd usage in DPDK?"
print('-------------------Instructor Embeddings------------------\n')
llm_response = qa_chain_instrucEmbed(query)
process_llm_response(llm_response)

-------------------Instructor Embeddings------------------

 Run the script ./run.sh in the DPDK-in-a-Box directory to start testpmd, which yields the testpmd prompt. Initialization consists of three steps: 1. EAL (Environment Abstraction Layer) —Find the number of cores and probe PCI devices; 2. Initialize the NICs; 3. Initialize the forwarding configuration. The runtime portion of testpmd is used to start packet forwarding.


Sources:
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf
/content/drive/MyDrive/PDFs/dpdk-cookbook-759202.pdf
